# Ollama Direct - No Custom Code

This uses Ollama's **native** OpenAI-compatible endpoint directly.

In [ ]:
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
print('✅ Installed')

In [ ]:
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
!ollama pull qwen2.5-coder:7b
print('\n✅ Ollama ready with qwen2.5-coder:7b')

In [ ]:
# Test Ollama's native OpenAI endpoint
import requests
r = requests.post('http://localhost:11434/v1/chat/completions', json={
    'model': 'qwen2.5-coder:7b',
    'messages': [{'role': 'user', 'content': 'Say hello'}]
})
print("Status:", r.status_code)
print("Response:", r.json())
if r.status_code == 200:
    print("\n✅ Ollama OpenAI API working!")

In [ ]:
# Tunnel directly to Ollama (port 11434)
import subprocess, re
from IPython.display import display, HTML

print("Starting tunnel to Ollama's native API...")
tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:11434'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''<div style="background:#9b59b6;padding:25px;border-radius:15px">
            <h2 style="color:white">🎯 Ollama Native API</h2>
            <p style="color:white;font-size:20px;font-family:monospace">{url}/v1</p>
            <hr style="border-color:rgba(255,255,255,0.3)">
            <p style="color:#eee">This is Ollama's built-in OpenAI endpoint - no custom code!</p>
            </div>'''))
            break

print('\n⚠️ Keep running!')
for line in tunnel.stdout:
    print(line, end='')